# Description


# Modules and Global Variables

In [1]:
from transformers import (
    AutoConfig, ElectraTokenizerFast, ElectraForTokenClassification, 
    DataCollatorForTokenClassification,
    TrainingArguments, Trainer,
)

from transformers.optimization import (
    AdamW, get_linear_schedule_with_warmup,
    Adafactor, AdafactorSchedule,
)

import torch
import wandb

import datasets
import evaluate

from sklearn.metrics import accuracy_score, f1_score

import numpy as np
import pandas as pd

import os
import re
import random

import demoji

/home/ubuntu/anaconda3/envs/jeonghyeon/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(f'torch.__version__: {torch.__version__}')
print(f'torch.cuda.is_available(): {torch.cuda.is_available()}')
NGPU = torch.cuda.device_count()
print(f'NGPU: {NGPU}')
# NGPU = torch.cuda.device_count()
# if NGPU > 1:
#     model = torch.nn.DataParallel(model, device_ids=list(range(NGPU)))

torch.__version__: 1.12.1
torch.cuda.is_available(): True
NGPU: 4


In [3]:
### labels

ce_labels = ['True', 'False']
pc_labels = ['positive', 'negative', 'neutral']
pc_binary_labels = ['True', 'False']
target_tagger_labels = ['Other', 'TRG_B', 'TRG_I']

labels = target_tagger_labels

label2id = {k: i for i, k in enumerate(labels)}
id2label = {i: k for i, k in enumerate(labels)}
num_labels = len(labels)

print(label2id)
print(id2label)

{'Other': 0, 'TRG_B': 1, 'TRG_I': 2}
{0: 'Other', 1: 'TRG_B', 2: 'TRG_I'}


In [4]:
### paths and names

PROJECT_NAME = 'target_tagging'
RUN_ID = 'uncleaned_v9'

DATA_V = 'uncleaned_v9'
DATA_T = 'tagger' # ce or pc or pc_binary or tagger
AUGMENTATION = False
AUG_NAME = 'balanced'

model_checkpoint = 'monologg/koelectra-base-v3-discriminator'

notebook_name = 'target_tagger_trainer.ipynb'

### fixed

model_name = re.sub(r'[/-]', r'_', model_checkpoint).lower()
run_name = f'{model_name}_{RUN_ID}'

ROOT_PATH = './'
SAVE_PATH = os.path.join(ROOT_PATH, 'training_results', run_name, 'target_tagger')
NOTEBOOK_PATH = os.path.join('./', notebook_name)

augornot = f'_{AUG_NAME}' if AUGMENTATION is True else ''
TRAIN_DATA_PATH = os.path.join(ROOT_PATH, 'dataset', DATA_V, f'{DATA_T}_train{augornot}.json')
EVAL_DATA_PATH = os.path.join(ROOT_PATH, 'dataset', DATA_V, f'{DATA_T}_dev.json')

!mkdir -p {SAVE_PATH}

In [5]:
if os.path.exists(SAVE_PATH):
    print(f'{SAVE_PATH} exists.')
else:
    print(f'{SAVE_PATH} does not exist.')
if os.path.exists(NOTEBOOK_PATH):
    print(f'{NOTEBOOK_PATH} exists.')
else:
    print(f'{NOTEBOOK_PATH} does not exist.')
if os.path.exists(TRAIN_DATA_PATH):
    print(f'{TRAIN_DATA_PATH} exists.')
else:
    print(f'{TRAIN_DATA_PATH} does not exist.')
if os.path.exists(EVAL_DATA_PATH):
    print(f'{EVAL_DATA_PATH} exists.')
else:
    print(f'{EVAL_DATA_PATH} does not exist.')

./training_results/monologg_koelectra_base_v3_discriminator_uncleaned_v9/target_tagger exists.
./target_tagger_trainer.ipynb exists.
./dataset/uncleaned_v9/tagger_train.json exists.
./dataset/uncleaned_v9/tagger_dev.json exists.


In [6]:
### rest of training args

report_to="wandb"

fp16 = False

num_train_epochs = 10
batch_size = 32
gradient_accumulation_steps = 1

optim = 'adamw_hf' # 'adamw_torch'

learning_rate = 3e-6 / 8 * batch_size * NGPU # 5e-5
weight_decay = 0.01 # 0
adam_epsilon = 1e-8

lr_scheduler_type = 'linear'
warmup_ratio = 0

save_total_limit = 2

load_best_model_at_end = True
metric_for_best_model ='f1_macro'

save_strategy = "epoch"
evaluation_strategy = "epoch"

logging_strategy = "steps"
logging_first_step = True 
logging_steps = 5

print(learning_rate)

4.8e-05


# WandB Configuration

In [7]:
%env WANDB_PROJECT={PROJECT_NAME}
%env WANDB_NOTEBOOK_NAME={NOTEBOOK_PATH}
%env WANDB_LOG_MODEL=true
%env WANDB_WATCH=all
wandb.login()

env: WANDB_PROJECT=target_tagging
env: WANDB_NOTEBOOK_NAME=./target_tagger_trainer.ipynb
env: WANDB_LOG_MODEL=true
env: WANDB_WATCH=all


wandb: Currently logged in as: dotsnangles. Use `wandb login --relogin` to force relogin


True

# Load Model, Tokenizer, and Collator

In [8]:
tokenizer = ElectraTokenizerFast.from_pretrained(model_checkpoint)
model = ElectraForTokenClassification.from_pretrained(
    model_checkpoint, label2id=label2id, id2label=id2label, num_labels=num_labels
)
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForTokenClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Some weights of ElectraForTokenClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
train_path = f'./dataset/{DATA_V}/raw_train.csv'
dev_path = f'./dataset/{DATA_V}/raw_dev.csv'
test_path = f'./dataset/{DATA_V}/raw_test.csv'
train = pd.read_csv(train_path)
dev = pd.read_csv(dev_path)
test = pd.read_csv(test_path)

### new
entity_property_pair = [
    '본품#가격', '본품#다양성', '본품#디자인', '본품#인지도', '본품#일반', '본품#편의성', '본품#품질',
    '브랜드#가격', '브랜드#디자인', '브랜드#인지도', '브랜드#일반', '브랜드#품질',
    '제품 전체#가격', '제품 전체#다양성', '제품 전체#디자인', '제품 전체#인지도', '제품 전체#일반', '제품 전체#편의성', '제품 전체#품질',
    '패키지/구성품#가격', '패키지/구성품#다양성', '패키지/구성품#디자인', '패키지/구성품#일반', '패키지/구성품#편의성', '패키지/구성품#품질'
]
sentiments = ['positive', 'negative', 'neutral']
target = ['Target']
special_tokens = ['&name&', '&affiliation&', '&social-security-num&', '&tel-num&', '&card-num&', '&bank-account&', '&num&', '&online-account&']
emojis = pd.concat([train.sentence_form, dev.sentence_form, test.sentence_form], ignore_index=True, verify_integrity=True).to_frame()
emojis = list(set(demoji.findall(' '.join(emojis.sentence_form.to_list())).keys()))
ep_labels = pd.Series(entity_property_pair, name='sentence_form', copy=True)

tokens2add = special_tokens + emojis + entity_property_pair + sentiments + target

print(len(tokenizer))
tokenizer_train_data = pd.concat([train.sentence_form, dev.sentence_form, test.sentence_form], ignore_index=True, verify_integrity=True).to_frame().drop_duplicates()
tokenizer_train_data = tokenizer_train_data.sentence_form.to_list()
new_tokenizer = tokenizer.train_new_from_iterator(tokenizer_train_data, vocab_size=1)
new_tokens = set(list(new_tokenizer.vocab.keys()) + tokens2add) - set(tokenizer.vocab.keys())
tokenizer.add_tokens(list(new_tokens))
print(len(new_tokenizer))
print(len(tokenizer))
model.resize_token_embeddings(len(tokenizer))

35000


3060
35283


Embedding(35283, 768)

In [10]:
print(len(new_tokens))
print(new_tokens)

283
{'듕', '&num&', '##㉦', '##앝', '##🕸', '👆', 'ღ', '🙆\u200d♂️', '펏', '본품#가격', '##˚', '🤡', 'ꈍ', '##잍', '😶', '✌️', '🕷', 'ˇ', '💪🏻', '🖐', '➰', '❣️', '👍🏻', '&name&', '😮', '💆', '##🧚', '##ɢ', '🍰', '&card-num&', '💇🏼\u200d♀️', '젔', '##읒', '&tel-num&', '👌🏻', '##ᴠ', '챳', '##®', '##🤮', '😺', '##닠', '##😶', '🌻', '💡', '⁉', '##👆', '제품 전체#품질', '본품#품질', 'neutral', 'ᴜ', '👏🏻', '##챦', '##➰', '퐉', '##👠', '##♪', '졓', '제품 전체#인지도', '뜌', '##👌', '🙋🏻', '☝️', '💆🏻\u200d♀️', '##옄', '컄', '##ᴘ', '##◍', '˚', '##💝', '핡', '제품 전체#다양성', '🎂', 'ᴛ', '패키지/구성품#품질', '챦', '##ˇ', '😡', '㉦', '##듕', '##🍎', '##ᴍ', '🍷', '##ᴏ', '##➕', '츌', '##ᴡ', '🕸', '##츌', '닠', '패키지/구성품#디자인', '##ᴛ', '💝', '🎀', '꺠', '̵', '🍼', '##☝', '🤘🏻', '잍', 'ෆ', '읒', '♥️', '##귯', '제품 전체#일반', '##ɪ', '웻', '🐥', '##💞', '##◡', '💬', '&bank-account&', '&social-security-num&', '☝🏻', '##♬', '##ɴ', '패키지/구성품#일반', '🧚', '😴', 'ɢ', '##❔', '패키지/구성품#가격', 'ᴇ', 'ғ', '◡', '‼️', '⏰', 'ᴠ', '##͈', '##💄', '##ꈍ', '♪', '〰️', '##촥', '💄', '🖒', 'ᴗ', '쨕', '##🤘', '😜', '닼', '👌', '##쵝', '🙆🏻', '##ᴜ', '

In [11]:
model.config.label2id, model.config.id2label, model.num_labels

({'Other': 0, 'TRG_B': 1, 'TRG_I': 2}, {0: 'Other', 1: 'TRG_B', 2: 'TRG_I'}, 3)

# Define Metric

In [12]:
accuracy_metric = evaluate.load('accuracy')
f1_metric = evaluate.load('f1')

In [13]:
# label_list = [0,1,2]

In [14]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [p for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_predictions = [x for true_prediction in true_predictions for x in true_prediction]
    true_labels = [
        [l for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [x for true_label in true_labels for x in true_label]
    
    accuracy = accuracy_metric.compute(references=true_labels, predictions=true_predictions)['accuracy']
    
    'Other', 'TRG_B', 'TRG_I'
    f1_other, f1_trg_b, f1_trg_i = tuple(f1_metric.compute(references=true_labels, predictions=true_predictions, average=None, labels=[0,1,2])['f1'])
    f1_macro = f1_metric.compute(references=true_labels, predictions=true_predictions, average='macro')['f1']
    f1_micro = f1_metric.compute(references=true_labels, predictions=true_predictions, average='micro')['f1']
    
    return {'accuracy': accuracy, 'f1_other': f1_other, 'f1_trg_b': f1_trg_b, 'f1_trg_i': f1_trg_i, 'f1_macro': f1_macro, 'f1_micro': f1_micro}

# Load Data

In [15]:
def preprocess_function(examples):
    input_ids = tokenizer.convert_tokens_to_ids(examples["input_tokens_list"])
    token_type_ids = [0 for _ in range(len(input_ids))]
    attention_mask = [1 for _ in range(len(input_ids))]   
    
    input_triplet = dict(
        input_ids = input_ids,
        token_type_ids = token_type_ids,
        attention_mask = attention_mask,
    )
    return input_triplet

In [16]:
train_dataset = pd.read_json(TRAIN_DATA_PATH)
eval_dataset = pd.read_json(EVAL_DATA_PATH)

In [17]:
train_dataset.input_tokens_list.to_list()
train_dataset.labels.to_list()
train_dataset = dict(
    input_tokens_list = train_dataset.input_tokens_list.to_list(),
    labels = train_dataset.labels.to_list(),
)

eval_dataset.input_tokens_list.to_list()
eval_dataset.labels.to_list()
eval_dataset = dict(
    input_tokens_list = eval_dataset.input_tokens_list.to_list(),
    labels = eval_dataset.labels.to_list(),
)

In [18]:
train_dataset = datasets.Dataset.from_dict(train_dataset) #.shuffle(seed=42)
eval_dataset = datasets.Dataset.from_dict(eval_dataset) #.shuffle(seed=42)
train_dataset = train_dataset.map(preprocess_function, batched=False)
eval_dataset = eval_dataset.map(preprocess_function, batched=False)

  0%|          | 0/4632 [00:00<?, ?ex/s]

 22%|██▏       | 1000/4632 [00:00<00:00, 7870.86ex/s]

 44%|████▎     | 2018/4632 [00:00<00:00, 9090.40ex/s]

 69%|██████▉   | 3192/4632 [00:00<00:00, 10228.94ex/s]

 93%|█████████▎| 4327/4632 [00:00<00:00, 10654.83ex/s]

100%|██████████| 4632/4632 [00:00<00:00, 10295.14ex/s]

  0%|          | 0/1159 [00:00<?, ?ex/s]

 48%|████▊     | 559/1159 [00:00<00:00, 2702.43ex/s]

100%|██████████| 1159/1159 [00:00<00:00, 4323.61ex/s]

In [19]:
train_dataset, eval_dataset

(Dataset({
     features: ['input_tokens_list', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 4632
 }),
 Dataset({
     features: ['input_tokens_list', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 1159
 }))

In [20]:
k = random.randrange(len(train_dataset))
print(train_dataset['input_ids'][k], '\n', train_dataset['labels'][k])
print(tokenizer.decode(train_dataset['input_ids'][k]), '\n', train_dataset['labels'][k])
print()
k = random.randrange(len(eval_dataset))
print(eval_dataset['input_ids'][k], '\n', eval_dataset['labels'][k])
print(tokenizer.decode(eval_dataset['input_ids'][k]), '\n', eval_dataset['labels'][k])

[2, 35178, 7, 10433, 4473, 16, 2075, 4516, 4505, 16, 2075, 4043, 4350, 2523, 7215, 4073, 6267, 4279, 12633, 2460, 5, 3] 
 [-100, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, -100]
[CLS] Target # 고수분, 고커버, 고광채 라 여름에 사용하기에 딱! [SEP] 
 [-100, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, -100]

[2, 35178, 7, 6470, 26284, 3243, 5152, 4070, 4050, 2374, 4073, 35023, 2741, 11177, 4086, 7309, 23413, 4152, 4956, 2024, 4153, 4129, 8566, 16456, 4116, 4150, 3] 
 [-100, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]
[CLS] Target # 사랑스러운 일룸가구 덕에 &name& 방 인테리어도 엄청 이뻐질것 같아서 벌써 설레네요 [SEP] 
 [-100, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]


# Load Trainer

In [21]:
args = TrainingArguments(
    output_dir=run_name,
    run_name=run_name,
    report_to=report_to,

    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,

    optim=optim,

    learning_rate=learning_rate,
    weight_decay=weight_decay,
    adam_epsilon=adam_epsilon,

    lr_scheduler_type=lr_scheduler_type,
    warmup_ratio=warmup_ratio,

    save_total_limit=save_total_limit,

    load_best_model_at_end=load_best_model_at_end,
    metric_for_best_model=metric_for_best_model,
    
    save_strategy=save_strategy,
    evaluation_strategy=evaluation_strategy,

    logging_strategy=logging_strategy,
    logging_first_step=logging_first_step, 
    logging_steps=logging_steps,
    
    fp16=fp16,
)

In [22]:
# es = EarlyStoppingCallback(early_stopping_patience=early_stopping_patience)

In [23]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    # callbacks=[es],
)

# Run Trainer

In [24]:
trainer.train()
wandb.finish()

The following columns in the training set don't have a corresponding argument in `ElectraForTokenClassification.forward` and have been ignored: input_tokens_list. If input_tokens_list are not expected by `ElectraForTokenClassification.forward`,  you can safely ignore this message.


/home/ubuntu/anaconda3/envs/jeonghyeon/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****


  Num examples = 4632


  Num Epochs = 10


  Instantaneous batch size per device = 32


...

Configuration saved in /tmp/tmp_hsr4556/config.json


Model weights saved in /tmp/tmp_hsr4556/pytorch_model.bin


tokenizer config file saved in /tmp/tmp_hsr4556/tokenizer_config.json


Special tokens file saved in /tmp/tmp_hsr4556/special_tokens_map.json


eval/accuracy,▁▇████████
eval/f1_macro,▁▇████████
eval/f1_micro,▁▇████████
eval/f1_other,▁▆█▇██████
eval/f1_trg_b,▁█████████
eval/f1_trg_i,▁▇████████
eval/loss,█▂▁▁▁▂▂▂▂▂
eval/runtime,▁▁▁▁▁▇▁█▁▁
eval/samples_per_second,█▇██▇▁█▁█▇
eval/steps_per_second,█▇██▇▁█▁█▇
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████


In [25]:
keep = [
    'added_tokens.json',
    'config.json',
    'pytorch_model.bin',
    'special_tokens_map.json',
    'tokenizer.json',
    'tokenizer_config.json',
    'vocab.txt'
]

ckpts = os.listdir(run_name)
for ckpt in ckpts:
    ckpt = os.path.join(run_name, ckpt)
    for item in os.listdir(ckpt):
        if item not in keep:
            os.remove(os.path.join(ckpt, item))

!mv wandb {run_name} {SAVE_PATH}/

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
